In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from tqdm import tqdm
import sys
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
import torch
from PIL import Image
import ast
sys.path.append('../input/tensorflow-great-barrier-reef')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile

In [ ]:
train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
train['pos'] = train.annotations != '[]'

In [ ]:
!mkdir -p ./yolo_data/fold1/images/val
!mkdir -p ./yolo_data/fold1/images/train
!mkdir -p ./yolo_data/fold1/labels/val
!mkdir -p ./yolo_data/fold1/labels/train

In [ ]:
fold = 1

annos = []
for i, x in train.iterrows():
    if x.video_id == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        if not x.pos: continue
        # val
    copyfile(f'../input/tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'./yolo_data/fold{fold}/images/{mode}/{x.image_id}.jpg')
    if not x.pos:
        continue
    r = ''
    anno = eval(x.annotations)
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'./yolo_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)

In [ ]:
data = '''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../yolo_data/fold1/  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['starfish']  # class names


# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''

In [ ]:
!git clone https://github.com/cv516Buaa/tph-yolov5.git

In [ ]:
%pip install timm

In [ ]:
with open('./tph-yolov5/data/reef_f1_naive.yaml', 'w') as fp:
    fp.write(data)

In [ ]:
%cd ./tph-yolov5

In [ ]:
!python train.py --img 1920 --batch 1 --epochs 6 --data reef_f1_naive.yaml --weights yolov5l.pt --name l6_3600_uflip_vm5_f1 --hyp data/hyps/hyp.scratch-high.yaml